# Import packages

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

# Load Data

In [2]:
df = pd.read_csv('data/raw_data.csv', index_col='user_id')
df = df.iloc[:, 3:]
df

,Gims,Kaaris,Maes,Booba,Ninho,Guy2bezbar,Leto,Koba la D,Damso,Freeze Corleone,Vald,Nekfeu,Zola,JUL,Niska,Sofiane,Fresh,Lomepal,Hamza,Laylow
user_id,,,,,,,,,,,,,,,,,,,,
1,2,3,3,2,5,5,4,5,5,3,4,4,5,1,5,3,3,1,3,3
2,2,2,3,2,3,1,3,2,4,3,3,5,3,2,4,3,1,3,3,5
3,2,2,4,3,3,1,3,2,5,5,4,5,4,4,4,4,1,2,2,4
4,2,3,3,2,4,3,4,4,2,4,5,4,5,1,2,2,1,3,2,4
5,2,2,3,3,5,3,4,3,4,5,3,4,3,2,4,3,3,1,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
136,1,1,2,1,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2
137,3,1,2,5,5,3,3,1,1,2,2,1,1,4,3,3,2,1,1,1


# Nearest user

In [3]:
df_distance_user = pd.DataFrame(
    squareform(pdist(df.values)),
    columns = df.index,
    index = df.index
)

In [4]:
def get_n_nearest_user(df_distance_user, id_user, n_user):
    distance_one_user = df_distance_user[id_user].sort_values().reset_index()[1:]
    distance_one_user.columns = ['user_id', 'distance']
    list_nearest_user = list(distance_one_user.iloc[:n_user]['user_id'])
    return list_nearest_user

In [5]:
get_n_nearest_user(df_distance_user, id_user=20, n_user=5)

[130, 5, 81, 63, 87]

In [6]:
df.loc[[20, 130, 5, 81, 63, 87]]

,Gims,Kaaris,Maes,Booba,Ninho,Guy2bezbar,Leto,Koba la D,Damso,Freeze Corleone,Vald,Nekfeu,Zola,JUL,Niska,Sofiane,Fresh,Lomepal,Hamza,Laylow
user_id,,,,,,,,,,,,,,,,,,,,
20,1,3,1,3,5,2,3,2,4,5,5,3,2,3,4,3,1,2,2,2
130,2,2,2,2,5,1,2,2,4,4,4,3,3,1,5,3,1,1,3,2
5,2,2,3,3,5,3,4,3,4,5,3,4,3,2,4,3,3,1,3,3
81,2,2,2,4,2,1,1,1,4,4,4,3,1,4,4,3,1,2,3,2
63,1,3,4,4,5,2,3,2,5,5,5,5,3,5,4,1,1,1,3,2
87,1,2,4,1,5,1,3,3,4,3,5,4,3,2,4,3,1,3,3,1


# Reco

In [7]:
list_all_favorite_artist = list(df[df >= 4].apply(lambda x: list(x.dropna().index), axis=1).values)
df['favorite_artist'] = [", ".join(i) for i in list_all_favorite_artist]

In [8]:
correlation_matrix = df.corr()

C:\Users\flore\AppData\Local\Temp\ipykernel_26652\3450543936.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


In [9]:
def get_reco_one_rappeur(correlation_matrix, rappeur, n_rappeur):
    return list(correlation_matrix[rappeur].sort_values(ascending=False)[1:n_rappeur+1].index)

In [13]:
def get_reco_rappeur_all_favorite(favorite_artist, correlation_matrix):
    if len(favorite_artist) > 0:
        rappeur_reco = []
        for rappeur in favorite_artist:
            rappeur_reco.extend(get_reco_one_rappeur(correlation_matrix, rappeur, 2))

        rappeur_reco = list(set(rappeur_reco))
        rappeur_reco = [rappeur for rappeur in rappeur_reco if rappeur not in favorite_artist]
        rappeur_reco = ", ".join(rappeur_reco)
    else:
        rappeur_reco = "Pas d'artiste à recommander"

    return rappeur_reco

In [14]:
reco = []
for favorite_artist in list_all_favorite_artist:
    reco.append(get_reco_rappeur_all_favorite(favorite_artist, correlation_matrix))

In [16]:
df['reco'] = reco

In [17]:
df[['favorite_artist', 'reco']]

,favorite_artist,reco
user_id,,
1,"Ninho, Guy2bezbar, Leto, Koba la D, Damso, Val...","Freeze Corleone, Laylow, Maes, Lomepal"
2,"Damso, Nekfeu, Niska, Laylow","Maes, Vald, Hamza, Zola, Ninho, Lomepal"
3,"Maes, Damso, Freeze Corleone, Vald, Nekfeu, Zo...","Leto, Lomepal, Hamza, Kaaris, Ninho"
4,"Ninho, Leto, Koba la D, Freeze Corleone, Vald,...","Maes, Lomepal, Hamza, Niska, Damso"
5,"Ninho, Leto, Damso, Freeze Corleone, Nekfeu, N...","Maes, Vald, Lomepal, Zola, Laylow"
...,...,...
135,,Pas d'artiste à recommander
136,,Pas d'artiste à recommander
137,"Booba, Ninho, JUL","Niska, Kaaris, Maes"


In [21]:
10/24/2022 15:08;"Un homme";"18-25 ans";"Entre...

In [37]:
pd.read_csv('bdd_gaspard_lili.csv', encoding = "latin", sep=';')

,Horodateur,Sexe,Age,Nombre film,Film 1 : MATRIX,Film 2 : STAR WARS : EPISODE VI - LE RETOUR DU JEDI,Film 3 - VERY BAD TRIP,Film 4 : Bienvenue chez les Ch'tis,Film 5 : AVATAR,Film 6 : Qu'est-ce qu'on a fait au Bon Dieu ?,...,Film 11 : LES EVADES,Film 12 : PSYCHOSE,Film 13 : LE VOYAGE DE CHIHIRO,Film 14 : ALIEN,Film 15 : TOY STORY,Film prefere,deuxieme film,genre film,deuxieme genre,genre film 2
0,24/10/2022 15:08,Un homme,18-25 ans,Entre 1 et 3,2,3,3,2,4,2,...,4,4,4,4,4,A la recherche du bonheur,Green Book,Drame,NaN,drame
1,24/10/2022 16:28,Un homme,18-25 ans,Plus de 5,3,4,3,3,5,2,...,5,2,4,3,4,Avengers,NaN,Science fiction,Action,NaN
2,24/10/2022 17:11,Une femme,18-25 ans,Entre 1 et 3,3,2,3,2,5,2,...,3,3,5,3,4,Azur & Asmar,NaN,Film d'animation,NaN,NaN
3,24/10/2022 17:18,Une femme,18-25 ans,Entre 1 et 3,1,2,4,4,1,4,...,4,1,1,1,3,Brice de Nice,NaN,Comedie,NaN,NaN
4,24/10/2022 15:51,Un homme,18-25 ans,0,2,3,4,4,5,4,...,3,3,3,3,4,Coach Carter,NaN,Comedie,Drame,NaN
5,24/10/2022 17:11,Un homme,25-30 ans,0,3,4,4,2,4,2,...,3,4,4,3,4,Coach Carter,NaN,Comedie,Drame,NaN
6,24/10/2022 15:02,Un homme,18-25 ans,Plus de 5,1,1,5,5,5,4,...,3,1,1,1,4,Divergente,NaN,Science fiction,Action,NaN
7,24/10/2022 15:04,Un homme,18-25 ans,Entre 1 et 3,3,4,3,2,4,1,...,5,3,4,1,3,Django Unchained,NaN,Drame,NaN,NaN
8,24/10/2022 14:51,Un homme,18-25 ans,Entre 1 et 3,3,4,4,3,3,3,...,5,2,2,2,3,Django Unchained,NaN,Drame,NaN,NaN
9,24/10/2022 15:18,Un homme,18-25 ans,Entre 1 et 3,4,2,5,4,5,4,...,4,1,1,2,3,Fatal,NaN,Comedie,NaN,NaN
